In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [87]:
df= pd.read_csv('horse.csv')

In [88]:
df.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,530101,38.5,66.0,28.0,cool,reduced,NaN,more_3_sec,...,45.0,8.4,NaN,NaN,died,no,11300,0,0,no
1,yes,adult,534817,39.2,88.0,20.0,NaN,NaN,pale_cyanotic,less_3_sec,...,50.0,85.0,cloudy,2.0,euthanized,no,2208,0,0,no
2,no,adult,530334,38.3,40.0,24.0,normal,normal,pale_pink,less_3_sec,...,33.0,6.7,NaN,NaN,lived,no,0,0,0,yes
3,yes,young,5290409,39.1,164.0,84.0,cold,normal,dark_cyanotic,more_3_sec,...,48.0,7.2,serosanguious,5.3,died,yes,2208,0,0,yes
4,no,adult,530255,37.3,104.0,35.0,NaN,NaN,dark_cyanotic,more_3_sec,...,74.0,7.4,NaN,NaN,died,no,4300,0,0,no


In [89]:
df.shape

(299, 28)

In [90]:
target= df[['outcome']]

In [91]:
target.value_counts()

outcome   
lived         178
died           77
euthanized     44
dtype: int64

In [92]:
features= df.drop('outcome',axis=1)

In [93]:
features

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,530101,38.5,66.0,28.0,cool,reduced,NaN,more_3_sec,...,distend_large,45.0,8.4,NaN,NaN,no,11300,0,0,no
1,yes,adult,534817,39.2,88.0,20.0,NaN,NaN,pale_cyanotic,less_3_sec,...,other,50.0,85.0,cloudy,2.0,no,2208,0,0,no
2,no,adult,530334,38.3,40.0,24.0,normal,normal,pale_pink,less_3_sec,...,normal,33.0,6.7,NaN,NaN,no,0,0,0,yes
3,yes,young,5290409,39.1,164.0,84.0,cold,normal,dark_cyanotic,more_3_sec,...,NaN,48.0,7.2,serosanguious,5.3,yes,2208,0,0,yes
4,no,adult,530255,37.3,104.0,35.0,NaN,NaN,dark_cyanotic,more_3_sec,...,NaN,74.0,7.4,NaN,NaN,no,4300,0,0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,yes,adult,533886,NaN,120.0,70.0,cold,NaN,pale_cyanotic,more_3_sec,...,distend_large,55.0,65.0,NaN,NaN,no,3205,0,0,no
295,no,adult,527702,37.2,72.0,24.0,cool,increased,pale_cyanotic,more_3_sec,...,distend_small,44.0,NaN,serosanguious,3.3,yes,2208,0,0,yes
296,yes,adult,529386,37.5,72.0,30.0,cold,reduced,pale_cyanotic,less_3_sec,...,distend_large,60.0,6.8,NaN,NaN,yes,3205,0,0,no
297,yes,adult,530612,36.5,100.0,24.0,cool,reduced,pale_pink,less_3_sec,...,distend_small,50.0,6.0,serosanguious,3.4,yes,2208,0,0,yes


In [94]:
features= pd.get_dummies(features)

In [95]:
features

,hospital_number,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,...,abdomen_firm,abdomen_normal,abdomen_other,abdomo_appearance_clear,abdomo_appearance_cloudy,abdomo_appearance_serosanguious,surgical_lesion_no,surgical_lesion_yes,cp_data_no,cp_data_yes
0,530101,38.5,66.0,28.0,NaN,45.0,8.4,NaN,11300,0,...,0,0,0,0,0,0,1,0,1,0
1,534817,39.2,88.0,20.0,NaN,50.0,85.0,2.0,2208,0,...,0,0,1,0,1,0,1,0,1,0
2,530334,38.3,40.0,24.0,NaN,33.0,6.7,NaN,0,0,...,0,1,0,0,0,0,1,0,0,1
3,5290409,39.1,164.0,84.0,5.0,48.0,7.2,5.3,2208,0,...,0,0,0,0,0,1,0,1,0,1
4,530255,37.3,104.0,35.0,NaN,74.0,7.4,NaN,4300,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,533886,NaN,120.0,70.0,NaN,55.0,65.0,NaN,3205,0,...,0,0,0,0,0,0,1,0,1,0
295,527702,37.2,72.0,24.0,NaN,44.0,NaN,3.3,2208,0,...,0,0,0,0,0,1,0,1,0,1
296,529386,37.5,72.0,30.0,NaN,60.0,6.8,NaN,3205,0,...,0,0,0,0,0,0,0,1,1,0
297,530612,36.5,100.0,24.0,NaN,50.0,6.0,3.4,2208,0,...,0,0,0,0,0,1,0,1,0,1


In [96]:
features.isnull().sum()

hospital_number                      0
rectal_temp                         60
pulse                               24
respiratory_rate                    58
nasogastric_reflux_ph              246
                                  ... 
abdomo_appearance_serosanguious      0
surgical_lesion_no                   0
surgical_lesion_yes                  0
cp_data_no                           0
cp_data_yes                          0
Length: 67, dtype: int64

In [97]:
x_train,x_test,y_train,y_test= train_test_split(features,target,train_size=0.75,random_state=100)

In [98]:
x_train.isnull().sum()

hospital_number                      0
rectal_temp                         47
pulse                               22
respiratory_rate                    47
nasogastric_reflux_ph              177
                                  ... 
abdomo_appearance_serosanguious      0
surgical_lesion_no                   0
surgical_lesion_yes                  0
cp_data_no                           0
cp_data_yes                          0
Length: 67, dtype: int64

In [99]:
x_test.isnull().sum()

hospital_number                     0
rectal_temp                        13
pulse                               2
respiratory_rate                   11
nasogastric_reflux_ph              69
                                   ..
abdomo_appearance_serosanguious     0
surgical_lesion_no                  0
surgical_lesion_yes                 0
cp_data_no                          0
cp_data_yes                         0
Length: 67, dtype: int64

In [100]:
simple_impute= SimpleImputer(missing_values=np.nan, strategy='mean')

In [101]:
x_train= simple_impute.fit_transform(x_train)
x_test= simple_impute.fit_transform(x_test)

In [102]:
pd.DataFrame(x_train).isnull().sum()

0     0
1     0
2     0
3     0
4     0
     ..
62    0
63    0
64    0
65    0
66    0
Length: 67, dtype: int64

In [103]:
pd.DataFrame(x_test).isnull().sum()

0     0
1     0
2     0
3     0
4     0
     ..
62    0
63    0
64    0
65    0
66    0
Length: 67, dtype: int64

# Voting

In [104]:
dec_tree=DecisionTreeClassifier(criterion='gini',splitter='best',max_depth=3)
log_reg=LogisticRegression()
knn=KNeighborsClassifier(n_neighbors=5)

In [105]:
voting=VotingClassifier(estimators=[('dec',dec_tree),('log',log_reg),('knn',knn)],voting='hard')

In [106]:
voting.fit(x_train,y_train)

C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/mo

VotingClassifier(estimators=[('dec', DecisionTreeClassifier(max_depth=3)),
                             ('log', LogisticRegression()),
                             ('knn', KNeighborsClassifier())])

In [107]:
y_pred=voting.predict(x_test)

In [108]:
y_pred

array(['died', 'lived', 'lived', 'lived', 'died', 'lived', 'lived',
       'lived', 'lived', 'euthanized', 'lived', 'lived', 'lived', 'lived',
       'lived', 'died', 'lived', 'lived', 'lived', 'died', 'lived',
       'lived', 'lived', 'lived', 'lived', 'lived', 'lived', 'lived',
       'lived', 'lived', 'lived', 'lived', 'lived', 'lived', 'lived',
       'lived', 'lived', 'lived', 'lived', 'euthanized', 'lived', 'lived',
       'lived', 'lived', 'lived', 'died', 'lived', 'lived', 'lived',
       'lived', 'lived', 'lived', 'lived', 'lived', 'lived', 'lived',
       'lived', 'lived', 'lived', 'lived', 'lived', 'lived', 'died',
       'lived', 'lived', 'lived', 'lived', 'lived', 'died', 'died',
       'died', 'lived', 'lived', 'lived', 'lived'], dtype=object)

In [109]:
accuracy_score(y_test,y_pred)

0.6933333333333334

In [110]:
for clas in dec_tree,log_reg,knn:
    clas .fit(x_train,y_train)
    y_pred=clas.predict(x_test)
    print(accuracy_score(y_test,y_pred))

0.7066666666666667
0.64


C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for 

0.5466666666666666


# Bagging

In [111]:
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier

In [112]:
bagging=BaggingClassifier(estimator=LogisticRegression(),n_estimators=2,bootstrap=True)

In [113]:
bagging.fit(x_train,y_train)

C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


BaggingClassifier(estimator=LogisticRegression(), n_estimators=2)

In [114]:
y_pred=bagging.predict(x_test)

In [115]:
y_pred

array(['lived', 'lived', 'lived', 'lived', 'lived', 'died', 'lived',
       'lived', 'lived', 'lived', 'lived', 'lived', 'lived', 'died',
       'lived', 'died', 'lived', 'lived', 'lived', 'lived', 'lived',
       'lived', 'lived', 'lived', 'lived', 'lived', 'lived', 'lived',
       'lived', 'lived', 'lived', 'lived', 'lived', 'lived', 'lived',
       'lived', 'lived', 'lived', 'lived', 'lived', 'lived', 'lived',
       'lived', 'lived', 'lived', 'died', 'lived', 'lived', 'lived',
       'lived', 'lived', 'lived', 'lived', 'lived', 'lived', 'died',
       'lived', 'lived', 'lived', 'died', 'lived', 'lived', 'lived',
       'lived', 'lived', 'died', 'lived', 'lived', 'lived', 'lived',
       'died', 'lived', 'lived', 'lived', 'lived'], dtype=object)

In [116]:
accuracy_score(y_test,y_pred)

0.6

In [117]:
params={'n_estimators':[1,2,3,4,5,6,7,8,9]}

In [118]:
grid=GridSearchCV(bagging,params)

In [119]:
grid.fit(x_train,y_train)

C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the 

GridSearchCV(estimator=BaggingClassifier(estimator=LogisticRegression(),
                                         n_estimators=2),
             param_grid={'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9]})

In [120]:
grid.best_params_

{'n_estimators': 7}

# Random Forest

In [121]:
random=RandomForestClassifier(n_estimators=100,bootstrap=False)

In [122]:
random.fit(x_train,y_train)

C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(bootstrap=False)

In [123]:
y_pred=random.predict(x_test)

In [124]:
y_pred

array(['died', 'lived', 'lived', 'lived', 'died', 'died', 'euthanized',
       'lived', 'lived', 'euthanized', 'lived', 'lived', 'euthanized',
       'died', 'lived', 'lived', 'died', 'lived', 'lived', 'died',
       'lived', 'lived', 'lived', 'lived', 'lived', 'lived', 'euthanized',
       'lived', 'lived', 'euthanized', 'lived', 'lived', 'died',
       'euthanized', 'died', 'lived', 'lived', 'died', 'died', 'lived',
       'lived', 'lived', 'lived', 'lived', 'lived', 'died', 'lived',
       'lived', 'lived', 'died', 'died', 'lived', 'euthanized', 'lived',
       'lived', 'died', 'died', 'lived', 'lived', 'died', 'lived',
       'lived', 'died', 'lived', 'lived', 'lived', 'lived', 'lived',
       'died', 'died', 'lived', 'lived', 'lived', 'lived', 'lived'],
      dtype=object)

In [125]:
accuracy_score(y_test,y_pred)

0.7066666666666667

# Boosting

### Adaboost

In [126]:
from sklearn.ensemble import AdaBoostClassifier

In [127]:
ada=AdaBoostClassifier(estimator=LogisticRegression(),n_estimators=20)

In [128]:
ada.fit(x_train,y_train)

C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iteration

AdaBoostClassifier(estimator=LogisticRegression(), n_estimators=20)

In [129]:
y_pred=ada.predict(x_test)

In [130]:
accuracy_score(y_test,y_pred)

0.6266666666666667

# Gradient Boosting

In [131]:
from sklearn.ensemble import GradientBoostingClassifier

In [132]:
gradient=GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,subsample=1.0)

In [133]:
gradient.fit(x_train,y_train)

C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier()

In [134]:
y_pred=gradient.predict(x_test)

In [135]:
accuracy_score(y_test,y_pred)

0.7333333333333333

In [136]:
from sklearn.preprocessing import LabelEncoder

In [137]:
lab=LabelEncoder()

In [138]:
y_train=lab.fit_transform(y_train)
y_test=lab.fit_transform(y_test)

C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Deepak\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [139]:
y_train

array([2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 1, 0, 1, 2, 2, 2,
       2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 1, 0, 2, 2, 2, 2, 2,
       2, 2, 0, 0, 0, 1, 1, 0, 2, 0, 0, 1, 2, 1, 1, 2, 1, 2, 2, 0, 2, 0,
       0, 0, 2, 0, 1, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 0, 0, 1, 2, 0,
       1, 2, 2, 2, 0, 0, 2, 1, 2, 2, 2, 0, 2, 2, 0, 1, 0, 2, 0, 0, 0, 2,
       1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       0, 1, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1,
       1, 2, 0, 2, 0, 2, 2, 2, 1, 2, 0, 1, 0, 0, 1, 2, 2, 0, 0, 2, 2, 1,
       2, 2, 1, 2, 1, 2, 0, 2, 0, 0, 2, 0, 0, 2, 1, 0, 1, 2, 2, 0, 2, 2,
       2, 0, 0, 0, 0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 1, 2, 2, 0, 2, 2, 2,
       2, 2, 0, 1])

In [140]:
y_test

array([2, 2, 2, 2, 0, 0, 1, 2, 2, 1, 2, 0, 2, 2, 2, 2, 0, 1, 2, 0, 2, 2,
       2, 2, 0, 1, 2, 2, 2, 1, 2, 1, 0, 2, 2, 2, 2, 0, 2, 2, 1, 0, 2, 1,
       2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2,
       2, 2, 0, 0, 2, 2, 2, 2, 1])

# XG Boost(Extreme Gradient Boosting)

In [141]:
import xgboost as xgb

In [146]:
xgb=xgb.XGBClassifier(n_estimators=100,max_depth=3,Learning_rate=0.1,max_depth_weight=2,subsample=1.0)

AttributeError: 'XGBClassifier' object has no attribute 'XGBClassifier'

In [147]:
xgb.fit(x_train,y_train)

C:\Users\Deepak\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [11:48:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "Learning_rate", "max_depth_weight" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(Learning_rate=0.1, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_depth_weight=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, ...)

In [148]:
y_pred=xgb.predict(x_test)

In [149]:
accuracy_score(y_test,y_pred)

0.6933333333333334